In [14]:
from wikibaseintegrator.models import Qualifiers, References, Reference
from wikibaseintegrator import WikibaseIntegrator
from wikibaseintegrator import wbi_login
from wikibaseintegrator import datatypes
from wikibaseintegrator.wbi_config import config as wbi_config
from os.path import join, isfile, exists, isdir, abspath, pardir
from os import listdir, getcwd
import pandas as pd

In [4]:
# Default variables
WDUSER, WDPASS = "Admin", "90J8XXXaO4Sr9^^Z"
# WDUSER, WDPASS = "Mohammadzainabbas", "fHh!%shFa6^h"

In [13]:
parent_dir = abspath(join(getcwd(), pardir))
data_dir = join(parent_dir, "data")
team_data = join(data_dir, "BDMA_Team_v1.csv")
project_data = join(data_dir, "BDMA_Projects_v1.csv")

In [2]:
def setup_config():
    """
    Set up WBI config to use local docker installation
    """
    wbi_config['MEDIAWIKI_API_URL'] = 'http://139.144.66.193:8181/api.php'
    wbi_config['SPARQL_ENDPOINT_URL'] = '"http://139.144.66.193:8282/proxy/wdqs/bigdata/namespace/wdq/sparql"'
    wbi_config['WIKIBASE_URL'] = 'http://wikibase.svc'

In [3]:
setup_config()

In [5]:
login = wbi_login.Login(user=WDUSER, password=WDPASS)

In [6]:
wbi = WikibaseIntegrator(login=login)

In [15]:
df = pd.from_csv(team_data)
df.head()

AttributeError: module 'pandas' has no attribute 'from_csv'